In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os

In [2]:
# Конфигурация подключения к PostgreSQL (база в Docker)
DB_CONFIG = {
    'host': 'localhost',  # или IP-адрес Docker контейнера
    'port': '5432',
    'database': 'server_metrics',  # имя вашей базы данных
    'user': 'postgres',
    'password': 'postgres'  # замените на ваш пароль
}

# Путь к Excel файлу
EXCEL_FILE_PATH = '../data/metrics.xlsx'